In [133]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
from glob import glob
from datetime import datetime
import sys
sys.path.append('/Users/w76m851/github_workspace/spicy-snow/')

from spicy_snow.processing.snow_index import calc_delta_cross_ratio, calc_delta_gamma, \
    clip_delta_gamma_outlier, calc_snow_index, calc_snow_index_to_snow_depth
from spicy_snow.processing.wet_snow import id_newly_wet_snow, id_wet_negative_si, \
    id_newly_frozen_snow, flag_wet_snow

def rmse(x, y):
    return np.sqrt((x - y)**2)

In [132]:
files = sorted(glob('/Users/w76m851/OneDrive - Montana State University/spicy_s1_stacks/*.nc'))
test = xr.open_dataset(files[0])
test

<xarray.Dataset>
Dimensions:         (x: 185, y: 144, band: 3, time: 58)
Coordinates:
  * x               (x) float64 -115.3 -115.3 -115.3 ... -115.1 -115.1 -115.1
  * y               (y) float64 44.34 44.34 44.34 44.34 ... 44.2 44.19 44.19
  * band            (band) object 'VV' 'VH' 'inc'
  * time            (time) datetime64[ns] 2019-08-05T01:35:54 ... 2020-03-02T...
    flight_dir      (time) object ...
    platform        (time) object ...
    relative_orbit  (time) float64 ...
    projection      int64 ...
Data variables: (12/17)
    s1              (time, band, y, x) float32 ...
    spatial_ref     int64 ...
    ims             (time, y, x) float64 ...
    fcf             (y, x) float64 ...
    deltaCR         (time, y, x) float32 ...
    deltaVV         (time, y, x) float32 ...
    ...              ...
    freeze_flag     (time, y, x) float32 ...
    wet_snow        (time, y, x) float32 ...
    perma_wet       (time, y, x) float32 ...
    lidar-sd        (y, x) float32 ...
    lidar-vh        (y, x) float32 ...
    lidar-dem       (y, x) float32 ...
Attributes:
    AREA_OR_POINT:      Point
    TIFFTAG_DATETIME:   2023:02:25 03:57:04
    TIFFTAG_SOFTWARE:   Created with GAMMA Software www.gamma-rs.ch data2geot...
    scale_factor:       1.0
    add_offset:         0.0
    resolution:         90
    s1_units:           dB
    site:               Banner
    site_abbrev:        USIDBS
    lidar-flight-time:  2020-02-18

In [134]:
A = np.arange(1, 3.1, 1)
B = np.arange(0, 1.01, 0.2)
C = np.arange(0, 1.001, 0.05)

result = xr.DataArray(np.empty((len(A), len(B), len(C)))*np.nan,
                      coords=(A, B, C))

dataset = test[['s1','deltaVV','ims','fcf','lidar-sd']]

td = abs(pd.to_datetime(dataset.time) - pd.to_datetime(dataset.attrs['lidar-flight-time']))
closest_ts_idx = np.where(td == td.min())[0][0]
closest_ts = dataset.time[closest_ts_idx]

print(datetime.now())

for a in A:
    ds = calc_delta_cross_ratio(dataset, A=a, inplace=False)
    for b in B:
        ds = calc_delta_gamma(ds, B=b, inplace=False)
        for c in C:
            print(f'A={a}; B={b}; C={c}')
            ds = clip_delta_gamma_outlier(ds)
            ds = calc_snow_index(ds)
            ds = calc_snow_index_to_snow_depth(ds, C=c, inplace=False)
            ds = id_newly_wet_snow(ds)
            ds = id_wet_negative_si(ds)
            ds = id_newly_frozen_snow(ds)
            ds = flag_wet_snow(ds)
            # Compare snow depths
            mask = ~(pd.isnull(ds['lidar-sd']) | pd.isnull(ds['snow_depth'].sel(time=closest_ts)) | 
                     ds['wet_snow'].sel(time=closest_ts).astype(bool))
            diff = ds['lidar-sd'].where(mask) - ds['snow_depth'].sel(time=closest_ts).where(mask)
            rmse = float(np.sqrt((diff**2).sum()/len(diff.values.flatten())))
            result.loc[a, b, c] = rmse
  
            

2023-03-02 17:58:40.985277
A=1.0; B=0.0; C=0.0
A=1.0; B=0.0; C=0.05
A=1.0; B=0.0; C=0.1
A=1.0; B=0.0; C=0.15000000000000002
A=1.0; B=0.0; C=0.2


KeyboardInterrupt: 

In [141]:
result = xr.DataArray(np.empty((len(A), len(B), len(C)))*np.nan,
                      coords=(A, B, C), dims=('A','B','C'))
result.loc[1, 0.4, 0.2]

<xarray.DataArray ()>
array(nan)
Coordinates:
    A        float64 1.0
    B        float64 0.4
    C        float64 0.2

In [107]:
ds['wet_snow'].sel(time=closest_ts)

<xarray.DataArray 'wet_snow' (y: 150, x: 193)>
array([[1., 1., 1., ..., 0., 1., 1.],
       [0., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 0., 1.],
       [1., 1., 1., ..., 0., 1., 0.],
       [1., 1., 1., ..., 1., 1., 0.]], dtype=float32)
Coordinates:
  * x               (x) float64 -115.3 -115.3 -115.3 ... -115.1 -115.1 -115.1
  * y               (y) float64 44.34 44.34 44.34 44.34 ... 44.2 44.19 44.19
    time            datetime64[ns] 2021-03-15T01:35:59
    flight_dir      object ...
    platform        object ...
    relative_orbit  float64 93.0
    projection      int64 ...

In [72]:
diff = test['lidar-sd'] - test['snow_depth'].sel(time=closest_ts)
diff.max()

<xarray.DataArray ()>
array(4.70286121)
Coordinates:
    projection      int64 0
    time            datetime64[ns] 2021-03-15T01:35:59
    flight_dir      object ...
    platform        object ...
    relative_orbit  float64 ...

In [95]:
test['wet_snow'] == test['wet_flag']

<xarray.DataArray (time: 97, y: 150, x: 193)>
array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
...
        ...,
        [ True,  True,  True, ...,  True, False,  True],
        [ True,  True,  True, ...,  True,  True, False],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True, False, False, ...,  True,  True,  True],
        [False, False, False, ..., False, False,  True],
        [False, False, False, ...,  True, False,  True],
        ...,
        [False, False,  True, ...,  True,  True, False],
        [False,  True, False, ...,  True,  True, False],
        [ True,  True, False, ...,  True,  True,  True]],

       [[False, False, False, ..., False,  True,  True],
        [ True, False, False, ...,  True,  True, False],
        [False, False, False, ...,  True,  True,  True],
        ...,
        [ True, False, False, ..., False, False,  True],
        [False,  True, False, ..., False, False, False],
        [False, False,  True, ...,  True,  True,  True]]])
Coordinates:
  * x               (x) float64 -115.3 -115.3 -115.3 ... -115.1 -115.1 -115.1
  * y               (y) float64 44.34 44.34 44.34 44.34 ... 44.2 44.19 44.19
  * time            (time) datetime64[ns] 2020-08-03T13:49:39 ... 2021-03-27T...
    flight_dir      (time) object 'descending' 'ascending' ... 'ascending'
    platform        (time) object 'S1A' 'S1A' 'S1B' 'S1B' ... 'S1B' 'S1B' 'S1B'
    relative_orbit  (time) float64 71.0 93.0 20.0 71.0 ... 93.0 20.0 71.0 93.0
    projection      int64 0